# Examples showcasing GEM EO commercial data collections

This is a notebook showcasing workflow examples for downloading EO data from commercial data collections available through Sentinel Hub.

### Data download examples in this notebook:

1.  PlanetScope 

2.  Pleiades

3.  Spot

4.  WorldView (+GeoEye)



In [1]:
%matplotlib inline
import datetime

import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np
from aenum import MultiValueEnum
from eolearn.core import EOPatch, FeatureType
from eolearn.io import SentinelHubDemTask, SentinelHubEvalscriptTask, SentinelHubInputTask, get_available_timestamps
from matplotlib.colors import BoundaryNorm, ListedColormap

from sentinelhub import CRS, Band, BBox, DataCollection, MimeType, SentinelHubRequest, SHConfig, Unit

### General parameters

In [45]:
# regions of interest

ps_bbox = BBox(bbox=[562218, 5174019, 564201, 5172501], crs=CRS.UTM_33N)
pleiades_bbox = wv_bbox = BBox(bbox=[562150.34, 5172481.37, 564214.37, 5174058.72], crs=CRS.UTM_33N)
spot_bbox = wv_bbox = BBox(bbox=[562150.34, 5172481.37, 564214.37, 5174058.72], crs=CRS.UTM_33N)
wv_bbox = BBox(bbox=[561924.43, 5172145.65, 564217.92, 5174392.08], crs=CRS.UTM_33N)

### 1. PlanetScope ( [PS](https://docs.sentinel-hub.com/api/latest/data/planet/planet-scope/) )

In [47]:
ps_data = DataCollection.define_byoc(
    "906b545b-4305-4968-b245-2abcb0609bc1",
    service_url="https://services.sentinel-hub.com",
    bands=[
        Band(name="Blue", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="Green", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="Red", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="NIR", units=(Unit.DN,), output_types=(np.uint16)),
    ],
)

In [48]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["Blue", "Green", "Red", "NIR"],
    output: [
        {
          id: "bands",
          bands: 4,
          sampleType: "UINT16"
        }
    ]
  };
}
function evaluatePixel(sample) {
  return [sample.Blue/10000, sample.Green/10000, sample.Red/10000, sample.NIR];
}
"""

In [49]:
ps_download_task = SentinelHubEvalscriptTask(
    features=(FeatureType.DATA, "bands"), evalscript=evalscript, data_collection=ps_data, resolution=30, max_threads=3
)

In [57]:
eopatch_ps = ps_download_task.execute(bbox=ps_bbox, time_interval=["2021-08-20", "2021-08-20"])

In [58]:
eopatch_ps

EOPatch(
  data={
    bands: numpy.ndarray(shape=(1, 51, 66, 4), dtype=uint16)
  }
  meta_info={
    size_x: 66
    size_y: 51
    time_difference: 1.0
    time_interval: ('2021-08-20T00:00:00', '2021-08-20T23:59:59')
  }
  bbox=BBox(((562218.0, 5172501.0), (564201.0, 5174019.0)), crs=CRS('32633'))
  timestamp=[datetime.datetime(2021, 8, 20, 9, 6, 12, 174000)]
)

### 2. [Pleiades](https://docs.sentinel-hub.com/api/latest/data/airbus/pleiades/)

In [37]:
pleiades_data = DataCollection.define_byoc(
    "934273b4-ffe6-464f-ac83-e49b58154e93",
    service_url="https://services.sentinel-hub.com",
    bands=[
        Band(name="B0", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="B1", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="B2", units=(Unit.DN,), output_types=(np.uint16)),
    ],
)

In [38]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B0", "B1","B2"],
    output: [
        {
          id: "bands",
          bands: 3,
          sampleType: "UINT16"
        }
    ]
  };
}
function evaluatePixel(sample) {
  return [sample.B0, sample.B1, sample.B2];
}
"""

In [39]:
pleiades_download_task = SentinelHubEvalscriptTask(
    features=(FeatureType.DATA, "bands"),
    evalscript=evalscript,
    data_collection=pleiades_data,
    resolution=5,
    max_threads=3,
)

In [43]:
eopatch_pleiades = pleiades_download_task.execute(bbox=pleiades_bbox, time_interval=["2017-09-01", "2017-09-28"])

In [44]:
eopatch_pleiades

EOPatch(
  data={
    bands: numpy.ndarray(shape=(1, 315, 413, 3), dtype=uint16)
  }
  meta_info={
    size_x: 413
    size_y: 315
    time_difference: 1.0
    time_interval: ('2017-09-01T00:00:00', '2017-09-28T23:59:59')
  }
  bbox=BBox(((562150.34, 5172481.37), (564214.37, 5174058.72)), crs=CRS('32633'))
  timestamp=[datetime.datetime(2017, 9, 13, 10, 3, 39, 100000)]
)

### 3. [Spot](https://docs.sentinel-hub.com/api/latest/data/airbus/spot/) 

In [3]:
spot_data = DataCollection.define_byoc(
    "57d54a21-09e9-40b7-b837-ff7c78b2b691",
    service_url="https://services.sentinel-hub.com",
    bands=[
        Band(name="B0", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="B1", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="B2", units=(Unit.DN,), output_types=(np.uint16)),
    ],
)

In [4]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B0", "B1","B2"],
    output: [
        {
          id: "bands",
          bands: 3,
          sampleType: "UINT16"
        }
    ]
  };
}
function evaluatePixel(sample) {
  return [sample.B1, sample.B2, sample.B3];
}
"""

In [5]:
spot_download_task = SentinelHubEvalscriptTask(
    features=(FeatureType.DATA, "bands"), evalscript=evalscript, data_collection=spot_data, resolution=6, max_threads=3
)

In [33]:
eopatch_spot = spot_download_task.execute(bbox=spot_bbox, time_interval=["2017-01-27", "2018-04-27"])

In [34]:
eopatch_spot

EOPatch(
  data={
    bands: numpy.ndarray(shape=(1, 263, 344, 3), dtype=uint16)
  }
  meta_info={
    size_x: 344
    size_y: 263
    time_difference: 1.0
    time_interval: ('2017-01-27T00:00:00', '2018-04-27T23:59:59')
  }
  bbox=BBox(((562150.34, 5172481.37), (564214.37, 5174058.72)), crs=CRS('32633'))
  timestamp=[datetime.datetime(2017, 5, 6, 9, 35, 12, 900000)]
)

### 4. [WorldView (+GeoEye)](https://docs.sentinel-hub.com/api/latest/data/maxar/world-view/) 

In [23]:
wv_data = DataCollection.define_byoc(
    "991fe3be-4d19-4d9f-9941-879da0a5c3b3",
    service_url="https://services.sentinel-hub.com",
    bands=[
        Band(name="Blue", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="Green", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="Red", units=(Unit.DN,), output_types=(np.uint16)),
        Band(name="NearIR1", units=(Unit.DN,), output_types=(np.uint16)),
    ],
)

In [24]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["Blue", "Green","Red", "NearIR1"],
    output: [
        {
          id: "bands",
          bands: 4,
          sampleType: "UINT16"
        }
    ]
  };
}
function evaluatePixel(sample) {
  return [sample.Blue, sample.Green, sample.Red, sample.NearIR1];
}
"""

In [25]:
wv_download_task = SentinelHubEvalscriptTask(
    features=(FeatureType.DATA, "bands"), evalscript=evalscript, data_collection=wv_data, resolution=5, max_threads=3
)

In [26]:
eopatch_wv = wv_download_task.execute(bbox=wv_bbox, time_interval=["2020-11-06", "2021-11-06"])

In [27]:
eopatch_wv

EOPatch(
  data={
    bands: numpy.ndarray(shape=(2, 449, 459, 4), dtype=uint16)
  }
  meta_info={
    size_x: 459
    size_y: 449
    time_difference: 1.0
    time_interval: ('2020-11-06T00:00:00', '2021-11-06T23:59:59')
  }
  bbox=BBox(((561924.43, 5172145.65), (564217.92, 5174392.08)), crs=CRS('32633'))
  timestamp=[datetime.datetime(2020, 11, 6, 9, 51, 16, 527000), ...], length=2
)